In [20]:
#MSA
from Bio.Seq import Seq
import Bio.SeqIO as SeqIO
import random

sick_patient_seq = SeqIO.read('HBS.fasta', 'fasta')
print(sick_patient_seq)
print(sick_patient_seq.seq)


# number of individuals in each generation 
POPULATION_SIZE = 100
SEQ_LEN = 15

ID: Sick
Name: Sick
Description: Sick patient HBB
Number of features: 0
Seq('ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGG...CAA', SingleLetterAlphabet())
ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGGTGCATCTGACTCCTGTGGAGAAGTCTGCCGTTACTGCCCTGTGGGGCAAGGTGAACGTGGATGAAGTTGGTGGTGAGGCCCTGGGCAGGCTGCTGGTGGTCTACCCTTGGACCCAGAGGTTCTTTGAGTCCTTTGGGGATCTGTCCACTCCTGATGCTGTTATGGGCAACCCTAAGGTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGACAACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACTGTGACAAGCTGCACGTGGATCCTGAGAACTTCAGGCTCCTGGGCAACGTGCTGGTCTGTGTGCTGGCCCATCACTTTGGCAAAGAATTCACCCCACCAGTGCAGGCTGCCTATCAGAAAGTGGTGGCTGGTGTGGCTAATGCCCTGGCCCACAAGTATCACTAAGCTCGCTTTCTTGCTGTCCAATTTCTATTAAAGGTTCCTTTGTTCCCTAAGTCCAACTACTAAACTGGGGGATATTATGAAGGGCCTTGAGCATCTGGATTCTGCCTAATAAAAAACATTTATTTTCATTGCAA


In [32]:
def generate_sequence(base, length=5, prob=2):
    p = ""
    a = ["A","C","G","T"]
    for i in range(0, length):
        r = random.randrange(0,4)
        #print(r)
        if r < prob:
            p += base[i]
        else:
            p += a[r]
    return p 

def generate_population(base):
    population = []
    for i in range(0, POPULATION_SIZE):
        population.append(generate_sequence(base, SEQ_LEN))
        #print(population[i])
    return population

base = 'ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGG'
population = generate_population(base)

In [34]:

def initial_movement(population, gap):
    if(gap > SEQ_LEN - 5):
        print("You have to take smaller gap")
        return
    initial_pop = []
    #for index, p in enumerate(population):
    for p in population:
        pom = ""
        move = random.randrange(0,gap)
        for i in range(0, move):
            pom += "-"
        new_p = pom + p
        new_p = new_p[0:SEQ_LEN]
        initial_pop.append(new_p)
        #print(new_p)
        #population[index] = new_p

    return initial_pop

initial_pop = initial_movement(population, 5)

In [ ]:
def fitness_function():
    

def msa(seq, seqs):
    

def initial_chromosome(initial_pop):
    